<a href="https://colab.research.google.com/github/zhouchun0105/credit-card-churners/blob/main/SUBMIT_creditcard_churners_team1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Credit card customer churn analysis and predictions**

Team 1, Afternoon Session
Ana Maharjan, Ann Lu, Chun Zhou, 

